### Import Libraries

In [ ]:
from pptx import Presentation
import win32com.client as win32
import pandas as pd
import numpy as np
from pathlib import Path
import pythoncom
import re
import sys 
win32c = win32.constants
import shutil
import os
from win32com import client
from win32com.client import constants
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN
from pptx.chart.data import CategoryChartData,XyChartData,BubbleChartData
import win32com.client
from pptx.dml.color import RGBColor
from pptx.util import Inches
from pptx.chart.data import ChartData
from pptx.enum.chart import XL_TICK_LABEL_POSITION
from pptx.enum.chart import XL_LABEL_POSITION
from win32com.client import constants as xl
from pptx.enum.chart import XL_CHART_TYPE
import pickle
from pptx.enum.dml import MSO_LINE,MSO_LINE_DASH_STYLE
import time
import itertools
from pptx.enum.chart import XL_AXIS_CROSSES,XL_LEGEND_POSITION
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict 
from io import BytesIO 
import math
from datetime import datetime
pythoncom.CoInitialize()  # Initialize COM

In [5]:
global themePath
themePath='..\Theme1.thmx'


# Genral Functions

### Cleaning DF

In [6]:
def cleaning_df(dfs_group,channels,tableBy=['Year'],flag=False,replacer={}):
    modified_dfs = {} 
    grand_totals = {}
    for table in dfs_group:
        df = dfs_group[table].copy()
        if ('Year' in df[df.columns[1]].unique())or('Year' in df[df.columns[2]].unique()) or (flag):
            df.iloc[7-1] = df.iloc[7-1].ffill().replace(np.nan,'')
            df.iloc[8-1] = df.iloc[8-1].ffill().replace(np.nan,'')
            seg=df[df.iloc[9-1][df.iloc[9-1].notna()].reset_index()['index'].unique()]
            restCol=list(set(df.columns)-set(seg.columns))+[df.columns[0]]
            sec_total=df[[col for col in df.columns if col in restCol]]
            seg.columns = (seg.iloc[8-1] + ' ' + seg.iloc[9-1].replace(np.nan, '') + ' '+ seg.iloc[7-1]).str.strip()            
            seg = seg.iloc[seg[seg[seg.columns[0]]==seg.columns[0]].index[0]+1:]
            sec_total.columns = (sec_total.iloc[7-1] + ' ' + sec_total.iloc[9-1].replace(np.nan, '')).str.strip()
            sec_total = sec_total.iloc[sec_total[sec_total[sec_total.columns[0]]==sec_total.columns[0]].index[0]+1:]
            sec_total=sec_total[[col for col in sec_total.columns if 'Total' not in col]]
        else:
            df.iloc[7-1] = df.iloc[7-1].ffill().replace(np.nan,'')
            seg=df[df.iloc[8-1][df.iloc[8-1].notna()].reset_index()['index'].unique()]
            restCol=list(set(df.columns)-set(seg.columns))+[df.columns[0]]
            sec_total=df[[col for col in df.columns if col in restCol]]
            seg.columns = (seg.iloc[7-1]  + ' '+ seg.iloc[8-1].replace(np.nan,'')).str.strip()
 
            seg = seg.iloc[seg[seg[seg.columns[0]]==seg.columns[0]].index[0]+1:]
            sec_total.columns = (sec_total.iloc[7-1]  + ' '+ sec_total.iloc[8-1].replace(np.nan,'')).str.strip()
            sec_total = sec_total.iloc[sec_total[sec_total[sec_total.columns[0]]==sec_total.columns[0]].index[0]+1:]
 
 
        seg = seg.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
        seg = seg.applymap(lambda x: 0 if pd.isna(x) else x)
        seg = seg.sort_values([col for col in seg.columns if 'Value Share' in col], ascending=False)
 
        sec_total = sec_total.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
        sec_total = sec_total.applymap(lambda x: 0 if pd.isna(x) else x)
        sec_total = sec_total.sort_values([col for col in sec_total.columns if 'Value Share' in col], ascending=False)
        i=0
        cols=seg.columns
        for group in channels:
            if len([col for col in cols if group in col])>0:
                i+=1
                groupdf=[col for col in cols if group in col]
                groupdf= tableBy+groupdf 
                groupdf=seg[groupdf]
                name=group+(' | '+table.split(' | ')[-1] if '|' in table else '')
                modified_dfs[name]=groupdf[(~groupdf[groupdf.columns[0]].astype(str).str.contains('Grand Total'))]
                grand_totals[name]=groupdf[(groupdf[groupdf.columns[0]].astype(str).str.contains('Grand Total'))]
                grand_totals[name].columns=grand_totals[name].columns.str.replace('Total','').str.replace(name,'').str.replace(group,'').str.strip()
                modified_dfs[name].columns=modified_dfs[name].columns.str.replace('Total','').str.replace(name,'').str.replace(group,'').str.strip()

        if i==0:
            for rep in replacer.keys():
                table=table.replace(rep,replacer[rep])
            modified_dfs[table]=seg[(~seg[seg.columns[0]].astype(str).str.contains('Grand Total'))]
 

        if i>1:
            modified_dfs[table]=sec_total[(~sec_total[sec_total.columns[0]].astype(str).str.contains('Grand Total'))]
            grand_totals[table]=sec_total[(sec_total[sec_total.columns[0]].astype(str).str.contains('Grand Total'))]
            grand_totals[table].columns=grand_totals[table].columns.str.replace('Total','').str.replace(table,'').str.replace(group,'').str.strip()
            modified_dfs[table].columns=modified_dfs[table].columns.str.replace('Total','').str.replace(table,'').str.replace(group,'').str.strip()

    for dic in [modified_dfs,grand_totals]:
        emptyDf=[key for key in dic.keys() if dic[key].shape[1]<=2]
        if emptyDf:
            columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
            for df in emptyDf:
                emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns))
                dic[df][emptyCol]=0.0
    return modified_dfs,grand_totals       
 

### Slide Duplication

#### takes 5 parameters 

##### 1 - index of the slide to be duplicated  can be list of integers and lists as [1,[2,3]] 
##### 2 - duplication : list of number ot duplication for each slide 
##### 3 - section_names :  list of strings of each section to be named should have the same length of duplication list 
##### 4 - path : the presentation path to duplicate from 
##### 5 - new_pre : the path to put the ouput presentation 

In [1]:
# import pythoncom
# import win32com.client as win32

# def slideDuplication(index=[0, 1], duplication=[1, 1], section_names=[''], path='', new_pre=''):
#     """
#     Duplicate slides in a PowerPoint presentation.

#     Parameters:
#     - index (list): List of slide indices to duplicate.
#     - duplication (list): Number of times each slide should be duplicated.
#     - section_names (list): Names for sections to be added.
#     - path (str): Path to the PowerPoint presentation file.
#     - new_pre (str): Path to save the duplicated presentation.

#     Returns:
#     - str: A message indicating success or failure.
#     """
#     pythoncom.CoInitialize()
    
#     if len(index) != len(duplication) or len(index) != len(section_names):
#         return 'The Index list is not equal to the Duplication or Section names list in length.'

#     try:
#         app = win32.Dispatch("PowerPoint.Application")
#         presentation = app.Presentations.Open(path)
#         lis = []  # Track the start slide index for each new section

#         for i in range(len(index)):
#             if isinstance(index[i], list):
#                 # If `index[i]` is a list of slide indices
#                 for _ in range(duplication[i]):
#                     for k in index[i]:
#                         slide = presentation.Slides.Item(k + 1)
#                         duplicated_slide = slide.Duplicate()
#                         duplicated_slide.MoveTo(presentation.Slides.Count)
#                 lis.append(presentation.Slides.Count - duplication[i] * len(index[i]) + 1)
#             else:
#                 # If `index[i]` is a single slide index
#                 slide = presentation.Slides.Item(index[i] + 1)
#                 for _ in range(duplication[i]):
#                     duplicated_slide = slide.Duplicate()
#                     duplicated_slide.MoveTo(presentation.Slides.Count)
#                 lis.append(presentation.Slides.Count - duplication[i] + 1)

#         # Add sections
#         for j in range(len(lis)):
#             if duplication[j] != 0:
#                 presentation.SectionProperties.AddBeforeSlide(lis[j], section_names[j])

#         # Remove default section if needed
#         if presentation.SectionProperties.Count > 1:
#             presentation.SectionProperties.Delete(1, True)

#         # Save and close
#         presentation.SaveAs(new_pre)
#         presentation.Close()
#         return "Slides duplicated and saved successfully."

#     except Exception as e:
#         return f"An error occurred: {e}"

#     finally:
#         app.Quit()
#         pythoncom.CoUninitialize()


In [8]:
####New_With_duplicate
import pythoncom

def slideDuplication(index=[0,1],duplication=[1,1],section_names=[''],path='',new_pre=''):
    """
    Duplicate slides in a PowerPoint presentation.

    Parameters:
    - index (list): List of slide indices to duplicate.
    - duplication (list): List specifying the number of times each slide should be duplicated.
    - section_names (list): List of names for sections to be added.
    - path (str): Path to the PowerPoint presentation file.
    - new_pre (str): Path to save the duplicated presentation.

    Returns:
    - str: A message indicating success or failure.
    """
    lis=[]
    if len(index)!=len(duplication)!=len(section_names):
        return 'The Index list not equal the Duplication number list in length'
    
    app = win32.Dispatch("PowerPoint.Application")
    presentation = app.Presentations.Open(path)
    
    # Iterate through the slides in the original presentation and copy them to the new presentation
    for i in range(len(index)):
        if type(index[i])==list:
            # If index is a list of slide indices
            for num_duplicate in range(duplication[i]):
                for k in index[i]:
                    slide=presentation.Slides[k]

                    duplicated_slide = slide.Duplicate()
                    duplicated_slide.MoveTo(presentation.Slides.Count)
                # Store the slide index of the new duplicated slide
                # duplicated_indices.append(presentation.Slides.Count)

                    # time.sleep(1)
            lis.append(presentation.Slides.count+1-(duplication[i]*len(index[i])))
        else:
            # If index is a single slide index
            slide=presentation.Slides[index[i]]
            for num_duplicate in range(duplication[i]):
                duplicated_slide = slide.Duplicate()
                duplicated_slide.MoveTo(presentation.Slides.Count)
            lis.append(presentation.Slides.count+1-duplication[i])

    # Add sections to the new presentation
    for j in range(len(lis)):
        if duplication[j]!=0:
            presentation.SectionProperties.AddBeforeSlide(lis[j],section_names[j] )
    # presentation.ApplyTheme(themePath)
    presentation.SectionProperties.Delete(1, True)
    presentation.SaveAs(new_pre)
    presentation.Close()
    # Close the original presentation and PowerPoint application
    app.Quit()
 


### createTableAndChart

#### takes 1 parameters 

##### 1 - shapes :  list of slide shapes

In [9]:
def createTableAndChart(shapes):
    """
    Extract tables and charts from a collection of shapes in a PowerPoint slide.

    Parameters:
    - shapes: A collection of shapes in a PowerPoint slide.

    Returns:
    - tables: A list of table shapes.
    - charts: A list of chart shapes.
    """
    charts = []
    tables = []
    for shape in shapes:
        if shape.has_chart:
            shape_id = shape.shape_id
            charts.append(shape)  # Add the chart shape to the charts list

        if shape.has_table:
            shape_id = shape.shape_id
            tables.append(shape)  # Add the table shape to the tables list
    return tables, charts


### get_shape_number

#### takes 2 parameters 

##### 1 - shapes :  list of slide shapes
##### 2 - text :  string to get it's index from the list of shapes 

In [10]:
def get_shape_number(shapes, text):
    """
    Get the index of a shape with specific text.

    Parameters:
    - shapes (list): A list of shapes in a slide.
    - text (str): The text to search for in the shapes.

    Returns:
    - int or None: The index of the shape if found, None otherwise.
    """
    for shape_index in range(len(shapes)):
        if shapes[shape_index].has_text_frame:  # Check if the shape has a text frame
            if shapes[shape_index].text == text:  # Compare the shape's text with the given text
                return shape_index  # Return the index of the shape if text matches
    return None  # Return None if the shape with the given text is not found

# Table Functions

In [11]:
#Hello World

### removeRowFromTable

#### takes 4 parameters 

##### 1 - table :  the table to delet row from
##### 2 - num_rows_to_remove :  integer number of rows to be deleted
##### 3 - table_height : unit in Inches, table height to remain the height of the table as befor. 
##### 4 - rowToExclude : number of rows to be excluded from removing or resizing basicly the header rows 


In [12]:
def removeRowFromTable(table,num_rows_to_remove,rowToExclude=1,height=0):
    """
    Remove rows from a table in a PowerPoint presentation and adjust row heights.

    Parameters:
    - table: The table object.
    - num_rows_to_remove (int): Number of rows to remove.
    - table_height (float): Desired height of the table.
    - rowToExclude (int): Index of the row to exclude.

    Returns:
    - table: The modified table object.
    """
    # Remove specified number of rows from the table
    if num_rows_to_remove:
        table_height = get_table_height(table) if height==0 else height
        for _ in range(num_rows_to_remove):
            if len(table.rows) > 1:  # Skip removing the first row if there is more than one row
                row = table.rows[rowToExclude]
                remove_row(table, row)

    #     table_height = Inches(table_height)  # Specify the desired table height
        tableHeightExclude=sum([table.rows[i].height for i in range(rowToExclude)])
        total_row_height = table_height - tableHeightExclude
        num_rows = len(table.rows)-rowToExclude  # Exclude the first row
        if num_rows > 0:
            cell_height = total_row_height / num_rows
            for row in range(rowToExclude, table.rows.__len__()):
                table.rows[row].height = int(cell_height)
    return table
# helper function

def remove_row(table, row):
    """
    Remove a row from a table in a PowerPoint presentation.

    Parameters:
    - table: The table object.
    - row: The row object to be removed.
    """
    tbl = table._tbl
    tr = row._tr
    tbl.remove(tr)

### col_cell_remove

#### takes 3 parameters 

##### 1 - table :  the table to delet columns from
##### 2 - num_columns_to_remove :  integer number of columns to be deleted
##### 3 - table_width : unit in Inches, table width to remain the width of the table as befor. 

In [13]:
def col_cell_remove(table, num_columns_to_remove):
    """
    Remove columns from a PowerPoint table and adjust the width of the remaining columns.

    Parameters:
    - table: The PowerPoint table object.
    - num_columns_to_remove (int): The number of columns to remove.

    Returns:
    - table: The modified PowerPoint table object after column removal.
    """
    for _ in range(num_columns_to_remove):
        col = table.columns[len(table.columns) - 1]  # Get the last column to remove
        table_width = get_table_width(table)  # Get the total width of the table
        remove_col(table, col)  # Remove the column from the table

        total_col_width = table_width - table.columns[0].width  # Calculate the total width of remaining columns
        num_columns = len(table.columns) - 1  # Exclude the first column
        if num_columns > 0:
            cell_width = total_col_width / num_columns  # Calculate the width for each remaining column
            for col_index in range(1, table.columns.__len__()):
                table.columns[col_index].width = int(cell_width)  # Set the width for each remaining column
    return table  # Return the modified table after column removal

def col_cell_remove_no_height_adjusted(table, num_columns_to_remove):
    """
    Remove columns from a PowerPoint table and DO NOT adjust the width of the remaining columns.

    Parameters:
    - table: The PowerPoint table object.
    - num_columns_to_remove (int): The number of columns to remove.

    Returns:
    - table: The modified PowerPoint table object after column removal.
    """
    for _ in range(num_columns_to_remove):
        col = table.columns[len(table.columns) - 1]  # Get the last column to remove
        table_width = get_table_width(table)  # Get the total width of the table
        remove_col(table, col)  # Remove the column from the table
    return table  # Return the modified table after column removal

## helper function

def remove_col(table, col):
    """
    Remove a column from a table in a PowerPoint presentation.

    Parameters:
    - table: The table object.
    - col: The column object to be removed.
    """
    col_idx = table._tbl.tblGrid.index(col._gridCol)
 
    for tr in table._tbl.tr_lst:
        tr.remove(tr.tc_lst[col_idx])
    table._tbl.tblGrid.remove(col._gridCol)


### delete_table_chart

#### takes 2 parameters 

##### 1 - slide :  the slide to delet chart or table from
##### 2 - element :  chart or table element to be remove from the slide can be found as chart._element


In [ ]:
def delete_table_chart(slide, element):
    flag = True
    # Iterate through the shapes in the slide
    for i in range(len(slide.shapes)):
        # Check if the specified element matches the current shape
        if element == slide.shapes[i]:
            shape = slide.shapes[i]
            # Remove the shape from the slide
            slide.shapes._spTree.remove(shape._element)

            flag = False
            break
    
    # If the specified element is not found in the shapes of the slide
    if flag:
        print("Shape is not a table or chart.")


### dfSort

#### takes 5 parameters 



Sorts the given dataframe based on sales data and specified parameters.

Parameters:

    df (DataFrame): The dataframe to be sorted.
    
    client_manuf (list of str): List of strings representing client brand or manufacturer to be included in the output         dataframe.
    
    col (str): String to identify which column to be selected (Brand or Manufacturer).
    
    num (int, optional): Number of rows to be included in the output dataframe. Default is 5.
    
    salesCol (str, optional): Column in which the dataframe is to be sorted on.

Returns:
    dict: A dictionary containing sorted dataframes.



In [ ]:
def dfSort(df, client_manuf, col, num=5, salesCol=''):
    """
    Sort and filter dataframes based on specified conditions.

    Parameters:
    - df (dict): Dictionary of dataframes.
    - client_manuf (list): List of client manufacturers.
    - col (str): Column to sort by.
    - num (int): Number of items to keep.
    - salesCol (str): Column indicating sales.

    Returns:
    - newDic (dict): Modified dictionary of dataframes.
    """
    newDic = {}  # Initialize an empty dictionary to store modified dataframes
    global ot  # Global variable to store 'other' data

    # Iterate through each dataframe in the dictionary
    for key in df.keys():
        if salesCol:  # Check if a sales column is specified
            sales = [column for column in df[key].columns if salesCol in column][0]  # Get the sales column name

        # Filter dataframe based on sales threshold and sort by sales
        topManuf = df[key][df[key][sales].astype(float) >= 0.01].sort_values(sales, ascending=False)

        # Exclude client manufacturers from the top manufacturers list
        topManuf = topManuf[(~topManuf[col].isin(client_manuf))].reset_index(drop=True)

        clientDf = df[key][df[key][col].isin(client_manuf)]
        clientDf = clientDf.sort_values(sales, ascending=False)

        # Define a list of common 'other' terms
        otherList = ["Other", "Other Manufacturer", "Others", "All Others", "Others Fabricante Unif.", "Others Marca Unif."]

        # Check if any of the 'other' terms are present in the top manufacturers list
        if any(otherValue in topManuf.iloc[:num - len(clientDf)][col].unique() for otherValue in otherList):
            topManuf = topManuf.iloc[:num - len(clientDf)]  # Filter out 'other' items
            topManuf = topManuf[~topManuf[col].str.contains("Other|Others|Other Manufacturer|All Others|Others Fabricante Unif.|Others Marca Unif.")]

        else:
            topManuf = topManuf.iloc[:num - len(clientDf) - 1]  # Exclude one extra row

        # Extract rows containing 'other' manufacturers
        other_rows = df[key][df[key][col].str.contains("Other|Others|Other Manufacturer|All Others|Others Fabricante Unif.|Others Marca Unif.")]

        # Extract client-specific data from the dataframe

        # Identify manufacturers not included in the dataframe
        # restClient = list(set(client_manuf) - set(clientDf[col].unique()))
        
        # # Append missing manufacturers to the dataframe
        # if restClient:
        #     clientDf = pd.concat([clientDf, pd.DataFrame({f'{col}': restClient}, columns=df[key].columns).replace(np.nan, 0)])

        # Sort client-specific data by sales

        try:
            total = pd.DataFrame()  # Initialize an empty dataframe to store 'other' data

            # Extract 'other' data from the dataframe
            other = df[key][(~df[key][col].isin(topManuf[col])) & (~df[key][col].isin(clientDf[col])) & (~df[key][col].isin(other_rows[col]))]
            other = pd.concat([other, other_rows])

            # Calculate mean and sum of 'other' data
            if other.shape[0] != 0:
                ot = other
                otMean = ot[[c for c in ot.columns if 'Price' in c]]
                otsum = ot[[c for c in ot.columns if 'Price' not in c]].drop(columns=col)
                otMean = pd.DataFrame(otMean.mean().values.reshape(1, -1), columns=otMean.columns)
                otsum = pd.DataFrame(otsum.sum().values.reshape(1, -1), columns=otsum.columns)
                otsum[col] = 'Others'
                otMean[col] = 'Others'
                total = otMean.merge(otsum)

        except:
            total = pd.DataFrame()  # In case of any exceptions, initialize an empty dataframe

        # Concatenate and sort dataframes, drop duplicates, and update the dictionary
        newDic[key] = pd.concat([topManuf, total]).sort_values(sales, ascending=False).drop_duplicates(subset=col)
        if len(clientDf)<num:
            newDic[key] = pd.concat([clientDf, newDic[key]])
        else:
    
            newDic[key] = clientDf
        newDic[key] = newDic[key].replace(np.nan,0)
        newDic[key] = newDic[key].sort_values(sales, ascending = False)

    return newDic  # Return the modified dictionary of dataframes


Change cell font_name and size and alignment

In [16]:
def set_cell_font(cell, font_name, font_size):
    """
    Set font style and size for text within a cell in a PowerPoint table.

    Parameters:
    - cell: The cell object.
    - font_name (str): Name of the font.
    - font_size (int): Size of the font in points.

    Returns:
    None
    """
    for paragraph in cell.text_frame.paragraphs:
        cell.text_frame.paragraphs[0].font.size = Pt(font_size)  # Set font size for the first paragraph
        cell.text_frame.paragraphs[0].font.name = font_name  # Set font name for the first paragraph
        for run in paragraph.runs:
            run.font.name = font_name  # Set font name for each run within the paragraph
            run.font.size = Pt(font_size)  # Set font size for each run within the paragraph
        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER  # Set alignment for the first paragraph


Get the base table height and width

In [17]:
def get_table_height(table):
    """
    Calculate the total height of a PowerPoint table.

    Parameters:
    - table: The PowerPoint table object.

    Returns:
    - float: Total height of the table.
    """
    total_height = sum([row.height for row in table.rows])  # Sum the heights of all rows
    return total_height


def get_table_width(table):
    """
    Calculate the total width of a PowerPoint table.

    Parameters:
    - table: The PowerPoint table object.

    Returns:
    - float: Total width of the table.
    """
    total_width = sum([col.width for col in table.columns])  # Sum the widths of all columns
    return total_width


Modifiy dataframes dictionary according to the number of shapes on slide

In [ ]:
# def splitDfs(dfDictionary, markets, splitList):
#     """
#     Splits the DataFrames in dfDictionary according to the given markets and splitList.

#     Parameters:
#     dfDictionary (dict): A dictionary where keys are strings representing market pairs and values are DataFrames.
#     markets (list): A list of markets to consider for splitting.
#     splitList (list): A list of integers representing the number of splits.

#     Returns:
#     list: A list of lists, where each inner list contains the split DataFrames for a market.
#     """
    
#     splitedDf = []  # Initialize the list to store split DataFrames
    
#     for market in markets:
#         # Find all keys in the dictionary that match the current market
#         #cat = [key for key in dfDictionary.keys() if (market == key.split(' | ')[0] or market == key.split(' | ')[1])]
#         cat = [key for key in dfDictionary.keys() if market in key.split(' | ')]
        
#         for split in range(len(splitList)):
#             # Calculate the start and end indices for the current split
#             start = sum(splitList[:split]) if split != 0 else split
#             end = sum(splitList[:split+1]) if split != 0 else splitList[split]
            
#             # Append the split DataFrames for the current range to the result list
#             splitedDf.append(cat[start:end])
    
#     return splitedDf


def genrateIndexList(scopeCategory, chartIndex=0, chartCount=4):
    """
    Generates a list of chart indices based on the scope category and chart parameters.

    Parameters:
    scopeCategory (list): A list representing the scope categories (e.g., sectors, segments, categories).
    chartIndex (int, optional): The starting index for the first chart. Default is 0.
    chartCount (int, optional): The number of charts to display on a full slide. Default is 4.

    Returns:
    list: A list containing the calculated indices for chart positioning.
    """
    
    # Return an empty list if the scopeCategory is empty
    if len(scopeCategory) == 0:
        return []
    
    # Calculate the number of pages needed
    page = len(scopeCategory)
    
    # Initialize the list to store chart indices
    lis = []
    
    # Loop to calculate indices until pages are exhausted
    while page > chartCount:
        lis.append(chartCount + chartIndex)
        page -= chartCount
    
    # Append the remaining charts to the list
    lis.append(page + chartIndex)
    
    return [lis]



In [ ]:
def splitDfs(dfDictionary, markets, splitList,area=[]):
    """
    Splits the DataFrames in dfDictionary according to the given markets and splitList.

    Parameters:
    dfDictionary (dict): A dictionary where keys are strings representing market pairs and values are DataFrames.
    markets (list): A list of markets to consider for splitting.
    splitList (list): A list of integers representing the number of splits.

    Returns:
    list: A list of lists, where each inner list contains the split DataFrames for a market.
    """
    
    splitedDf = []  # Initialize the list to store split DataFrames
    
    for market in markets:
        # Find all keys in the dictionary that match the current market
        # cat = [key for key in dfDictionary.keys() if (market == key.split(' | ')[0] or market == key.split(' | ')[1])]
        # print(dfDictionary.keys())
        cat = [key for key in dfDictionary.keys() if market in key.split(' | ')]
        if area:
            cat = [key for key in cat  if key.split(' | ')[1] in area]

        for split in range(len(splitList)):
            # print(split)
            # Calculate the start and end indices for the current split
            start = sum(splitList[:split]) if split != 0 else split
            end = sum(splitList[:split+1]) if split != 0 else splitList[split]
            
            # Append the split DataFrames for the current range to the result list
            splitedDf.append(cat[start:end])
    
    return splitedDf

In [19]:
def splitDfsPromo(dfDictionary, markets, splitList):
    """
    Splits the DataFrames in dfDictionary according to the given markets and splitList.

    Parameters:
    dfDictionary (dict): A dictionary where keys are strings representing market pairs and values are DataFrames.
    markets (list): A list of markets to consider for splitting.
    splitList (list): A list of integers representing the number of splits.

    Returns:
    list: A list of lists, where each inner list contains the split DataFrames for a market.
    """
    
    splitedDf = []  # Initialize the list to store split DataFrames
    
    for market in markets:
        # Find all keys in the dictionary that match the current market
        #cat = [key for key in dfDictionary.keys() if (market == key.split(' | ')[0] or market == key.split(' | ')[1])]
        cat = [key for key in dfDictionary.keys() if market in key.split(' | ')]
        lis = splitList if len(cat)==1 else splitList * len(cat) 
        for split in range(len(lis)):
            # Calculate the start and end indices for the current split
            start = sum(lis[:split]) if split != 0 else split
            end = sum(lis[:split+1]) if split != 0 else lis[split]
            
            # Append the split DataFrames for the current range to the result list
            if len(cat[start:end]) >0:
                splitedDf.append(cat[start:end])
    
    return splitedDf


Set the format to the secondary value axis (PPA ISD slide)

In [20]:

def secondaryValueAxes(chart, formatVal='#,##0.00 K\r'):
    """
    Updates the number format of the secondary value axis in a chart.

    Parameters:
    chart : Chart object
        The chart object whose secondary value axis number format needs to be updated.
    formatVal : str, optional
        The new number format code to be applied to the secondary value axis. 
        Default is '#,##0.00 K\r'.

    Notes:
    - This function assumes the chart has a secondary value axis.
    - The function will modify the number format of the first found secondary value axis.
    """
    
    # Access the part and XML element of the chart
    chart_part = chart.part
    chart_element = chart.element

    # Define the XML namespaces
    namespaces = {
        'c': 'http://schemas.openxmlformats.org/drawingml/2006/chart',
        'a': 'http://schemas.openxmlformats.org/drawingml/2006/main'
    }

    # Find the plot area within the chart XML
    plot_area = chart_element.find('.//c:plotArea', namespaces)
    secondary_value_axes = plot_area.findall('.//c:valAx', namespaces)

    # Check if there are more than one value axes (indicating a secondary axis exists)
    if len(secondary_value_axes) > 1:
        secondary_value_axis = secondary_value_axes[1]  # Assuming the second valAx is the secondary axis

        # Find the number format element within the secondary value axis
        num_fmt = secondary_value_axis.find('.//c:numFmt', namespaces)
        if num_fmt is not None:
            # Modify the number format code
            num_fmt.set('formatCode', formatVal)
            # Optional: Ensure the format is linked to the source
            # num_fmt.set('sourceLinked', '1')
        else:
            print("No number format found for the secondary value axis.")
    else:
        print("No secondary value axis found.")

        


In [21]:
def selectClientBrands(df, col, sortcol):
    """
    Filter DataFrame rows based on client brands and value share.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - col (str): The column name containing the brands to filter by.
    - sortcol (str): The column name by which to sort the filtered DataFrame.

    Returns:
    - pd.DataFrame: Filtered DataFrame sorted by the specified column.
    """
    filtered_rows = []
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Check if any client brand is present in the specified column and the value in sortcol is greater than 0
        if any(brand == row[col] for brand in client_brands) and row[sortcol] > 0:
            filtered_rows.append(row)
    
    # Create a DataFrame from the filtered rows
    result = pd.DataFrame(filtered_rows)
    
    # Sort the result DataFrame by the specified column if it's not empty
    if not result.empty:
        result = result.sort_values(by=sortcol, ascending=False).reset_index(drop=True)
    
    return result


format_number : format the number to be 2 digit number after the point (2.00) , Add ' every three number start from the right , Add the currency symbol after or before the number

In [ ]:
def format_number(number, use_decimals=False, decimals=0, use_apostrophes=False, currency_symbol=None, currency_before=True):
    """
    Format the number with specified options for decimals, apostrophes, and currency symbol.

    Args:
    - number (float, int, or str): The number to be formatted.
    - use_decimals (bool): If True, format the number to have the specified number of decimal places.
    - decimals (int): The number of decimal places to include (if use_decimals is True).
    - use_apostrophes (bool): If True, format the number with an apostrophe every three digits.
    - currency_symbol (str): The currency symbol to include (e.g., '$', '€').
    - currency_before (bool): If True, place the currency symbol before the number; otherwise, after.

    Returns:
    - str: The formatted number as a string.
    """
    # Convert string input to float or int
    if isinstance(number, str):
        try:
            number = float(number)
        except ValueError:
            return "Invalid input"

    if use_decimals:
        if decimals == 0:
            formatted_number = f"{number:.0f}"
        else:
            formatted_number = f"{number:.{decimals}f}"

    if not use_decimals:
        formatted_number = f"{int(number+0.5):,}"  # Always format with two decimal places

    if use_apostrophes:
        formatted_number = formatted_number.replace(',', "'")

    if currency_symbol:
        if currency_before:
            formatted_number = f"{currency_symbol} {formatted_number}"
        else:
            formatted_number = f"{formatted_number} {currency_symbol}"

    return formatted_number

# Example usage:
#number = "1234567890.1"

# Use decimal formatting and apostrophes
#print(format_number(number, use_decimals=False, decimals=0, use_apostrophes=True,currency_symbol='$', currency_before=True))  # Output: 1'234'567'890.10


In [23]:
def remove_specific_table_from_slide(slide, table_index):
    table_count = 0
    for shape in slide.shapes:
        if shape.has_table:
            if table_count == table_index:
                sp = shape._element
                sp.getparent().remove(sp)
                break
            table_count += 1

In [24]:
def count_df(data, lis):
    matching_keys = [key for category in lis for key in data.keys() if category in key.split(' | ')]        
    return matching_keys

In [25]:
def combine_duplications(Scope,count_df, data_frames):
    combined_duplications = []
    for data_frame in data_frames:
        if data_frame:
            duplications = [len(count_df(data_frame, value if isinstance(value, list) else [value])) for key, value in Scope.items()]
            combined_duplications.extend(duplications)
        else:
            duplications = [*[0]*5]
            combined_duplications.extend(duplications)
    return combined_duplications

In [26]:
def DetectHeader(df):
    # Step 0: Skip the first 10 rows
    df_skipped = df.iloc[13:].reset_index(drop=True)
    vals = df_skipped.values
   
    # Step 1: Identify rows where all elements are strings
    is_string_row = [all(isinstance(i, str) for i in row) for row in vals]
   
    # Case 1: Check if all rows are not string rows
    if np.all([not val for val in is_string_row]):
        value_column2 = df_skipped.iloc[:, 1].values  # Selecting the second column
        for idx, value in enumerate(value_column2):
            try:
                # Try converting the value to an int
                converted_value = int(value)
                df_skipped.iloc[idx + 1] = (
                df_skipped.iloc[idx + 1].replace(np.nan, "").astype(str).str.strip() + ' ' +
                df_skipped.iloc[idx].ffill().replace(np.nan, "").astype(str).str.strip()
            )
            except ValueError:
                # If conversion fails, check if 'value' contains the substrings
                if "Volume" in str(value) or "Value Sales" in str(value):
                    df_skipped.iloc[idx + 1] = (
                    df_skipped.iloc[idx].ffill().replace(np.nan, "").astype(str).str.strip() + ' ' +
                    df_skipped.iloc[idx + 1].replace(np.nan, "").astype(str).str.strip())
                else:
                    continue
            break
 
    # Recheck string rows after modifications
    is_string_row = [all(isinstance(i, str) for i in row) for row in df_skipped.values]
    break_point = None
    for i, v in enumerate(is_string_row):
        if v:  
            break_point = i
            break
 
    if break_point is not None:
        df_skipped.columns = df_skipped.iloc[break_point].str.strip()
        df_skipped = df_skipped.iloc[break_point + 1:].reset_index(drop=True)  
    return df_skipped    

In [27]:

# def DetectHeader(df):
#     # Step 0: Skip the first 14 rows to start with data-like rows
#     df_skipped = df.iloc[14:].reset_index(drop=True)
   
#     vals = df_skipped.values
#     # Step 1: Identify rows where all elements are strings
#     is_string_row = [all(isinstance(i, str) and i for i in row) for row in vals]
 
#     # Case 1: Check if all rows are not string rows
#     if np.all([not val for val in is_string_row]):
#         value_column2 = df_skipped.iloc[:, 2].values  # Selecting the third column here
#         for idx, value in enumerate(value_column2):
#             try:
#                 # Attempt to convert the value to int, indicating it's a non-header row
#                 int(value)
#                 # Concatenate the previous row as headers if integer conversion works
#                 df_skipped.iloc[idx + 1] = (
#                     df_skipped.iloc[idx + 1].replace("", "").astype(str).str.strip() + ' ' +
#                     df_skipped.iloc[idx].ffill().replace("", "").astype(str).str.strip()
#                 )
#             except ValueError:
#                 if "P3M" in str(value):
#                     df_skipped.iloc[idx + 1] = (
#                         df_skipped.iloc[idx + 1].replace("", "").astype(str).str.strip() + ' ' +
#                         df_skipped.iloc[idx].ffill().replace("", "").astype(str).str.strip()
#                     )
#                 elif "Volume" in str(value) or "Value Sales" in str(value):
#                     df_skipped.iloc[idx + 1] = (
#                         df_skipped.iloc[idx].ffill().replace("", "").astype(str).str.strip() + ' ' +
#                         df_skipped.iloc[idx + 1].replace("", "").astype(str).str.strip()
#                     )
#                 else:
#                     continue
#             break
 
#     # Recheck string rows after modifications
#     is_string_row = [all(isinstance(i, str) and i for i in row) for row in df_skipped.values]
#     break_point = None
#     for i, v in enumerate(is_string_row):
#         if v:  
#             break_point = i
#             break
 
 
#     if break_point is not None:
#         cleaned_columns = [
#             ' '.join(str(col).split()).replace("nan", "").strip()  # Remove "nan" and extra spaces
#             for col in df_skipped.iloc[break_point]
#         ]
#         df_skipped.columns = cleaned_columns
#         # df_skipped.columns = df_skipped.iloc[break_point].str.strip()
#         df_skipped = df_skipped.iloc[break_point + 1:].reset_index(drop=True)  
#     return df_skipped    

In [28]:
import win32com.client
import pythoncom
from pptx import Presentation

def open_chart_data_in_excel(pptx_path, output_pptx_path, OpenEditData = True):
    """Open PowerPoint and Excel workbooks associated with charts and close them."""
    # Initialize COM for PowerPoint and Excel
    if not OpenEditData:
        print("Stopping the script as 'OpenEditData' is set to False.")
        sys.exit()  # Terminate the script here
    else :
        powerpoint = win32com.client.Dispatch("PowerPoint.Application")
        excel = win32com.client.Dispatch("Excel.Application")

        try:
            # Open the PowerPoint file using COM
            ppt_pres = powerpoint.Presentations.Open(pptx_path, WithWindow=False)
        
            # Loop through the slides and shapes to find charts
            for slide in ppt_pres.Slides:
                slide_index = slide.SlideIndex

                for shape in slide.Shapes:
                    if shape.HasChart:
                        chart = shape.Chart
                        # Check if the chart type is bubble (15 is the enumeration for xlBubble)
                        if chart.ChartType in [15, -4169]:
                            chart_data = chart.ChartData
                            # Activate the workbook
                            workbook = chart_data.Workbook
                            workbook.Application.Visible = False
                            print(f"Slide {slide_index}: Opened Excel workbook: {workbook.FullName}")
                            
                            # Close the workbook after processing
                            workbook.Close(SaveChanges=False)  # Close the workbook without saving
                            
            # Save the PowerPoint file if needed
            ppt_pres.SaveAs(output_pptx_path)
        
        finally:
            # Close only the specific presentation
            ppt_pres.Close()
            
            # Release COM objects explicitly
            del ppt_pres
            
            # Optionally release other COM objects if no longer needed
            # You can keep the PowerPoint application running if you need to open other presentations later
            # del powerpoint
            # del excel
            
            # Uninitialize COM
            pythoncom.CoUninitialize()


In [29]:
def RenameSections(outputPath,oldname="",newname=""): 
    app = win32.Dispatch("PowerPoint.Application")
    # Open the presentation
    prsSection = app.Presentations.Open(outputPath, WithWindow=False)

    # Iterate through the sections (section indices start from 1)
    for sectionNum in range(1, prsSection.SectionProperties.Count + 1):
        # Rename each section by replacing part of its name
        sectionName = prsSection.SectionProperties.Name(sectionNum).replace(
           oldname,newname)
        prsSection.SectionProperties.Rename(sectionNum, sectionName)

    # Save and close the presentation
    prsSection.SaveAs(outputPath)
    prsSection.Close()

In [30]:
def mround_numpy(x, base):
    return np.round(x / base) * base

In [ ]:
def dropemptydf(dic):
    keys_to_remove = [k for k, v in dic.items() if v.empty]
    for k in keys_to_remove:
        del dic[k]